In [1]:
from glob import glob

In [7]:
r = glob('rationales/FT3/batch_*')
print(len(r))

14171


In [6]:
print(len(glob('outputs/FT3/attentions/batch_*.pt')))

14345


In [16]:
from datasets import load_from_disk
ra = load_from_disk('rationales/FT3/batch_9')

In [24]:
ra[13]

{'Rating': 0,
 'Rationale': '불친절하고',
 'Unrationale': '매우 카페에 와이파이가 안됨. 그리고 맛이 앖음..'}